<a href="https://colab.research.google.com/github/kalz2q/mycolabnotebooks/blob/master/learnelixir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# メモ

elixir を齧る。かじる。

今のイメージ $\quad$ erlang 上で、erlang は 並行処理のためのシステムで、その erlang 上で理想的な言語を作ろうとしたら、ruby + clojure みたいな言語になった。

Dave Thomas と まつもとゆきひろ が勧めているのだからいい言語なのだろう。


* https://elixirschool.com/ja/lessons/basics/control-structures/
* https://magazine.rubyist.net/articles/0054/0054-ElixirBook.
* https://dev.to/gumi/elixir-01--2585
* https://elixir-lang.org/getting-started/introduction.html

---
本を買った。

プログラミング elixir

dave thomas, 笹田耕一・鳥居雪訳、 ohmsha
programming elixir |> 1.6

を読む。



In [14]:
%%capture
!wget https://packages.erlang-solutions.com/erlang-solutions_2.0_all.deb && sudo dpkg -i erlang-solutions_2.0_all.deb
!sudo apt update
!sudo apt install elixir

In [ ]:
!elixir -v
!date

Erlang/OTP 24 [erts-12.2.1] [source] [64-bit] [smp:2:2] [ds:2:2:10] [async-threads:1] [jit]

Elixir 1.13.0 (compiled with Erlang/OTP 24)
Wed Mar 16 16:43:56 UTC 2022


---
メモ

`!elixir -h` (ヘルプ)としたらシェルワンライナー `elixir -e` が使えるらしいことがわかった。

`iex` というのがインタラクティブ環境なのだが、colab では使いにくいので `elixir -e` で代用する。

In [ ]:
!elixir -e 'IO.puts 3 + 3'
!elixir -e 'IO.puts "hello world!"'

6
hello world!


In [ ]:
# 次のようにすればファイルが作れる
%%writefile temp.exs
IO.puts "this is a pen."

Writing temp.exs


In [ ]:
# cat してみる
!cat temp.exs

IO.puts "this is a pen."

In [ ]:
# ファイルを elixir で実行する
!elixir temp.exs

this is a pen.


---
ネットで紹介されていた次のコードセルのコードはどうやって実行するのだろう。 今はわからなくていいと思うがとりあえず転記しておく。

説明:

このプログラムでは、Parallel というモジュールに pmap という関数を定義しているmap は、与えられたコレクションに対して map（Ruby での Enumerable#map と同じようなものと考えて下さい）を行なうのですが、 各要素の処理を、要素数の分だけプロセスを生成し、各プロセスで並行に実行する、というものです。 ちょっと見ても、よくわからないような気がしますが、大丈夫、本書を読めば、わかるようになりる

とのこと。

In [ ]:
%%writefile temp.exs
defmodule Parallel do
  def pmap(collection, func) do
    collection
    |> Enum.map(&(Task.async(fn -> func.(&1) end)))
    |> Enum.map(&Task.await/1)
  end
end

result = Parallel.pmap 1..1000, &(&1 * &1)

IO.inspect result

Overwriting temp.exs


In [ ]:
!elixir temp.exs

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324,
 361, 400, 441, 484, 529, 576, 625, 676, 729, 784, 841, 900, 961, 1024, 1089,
 1156, 1225, 1296, 1369, 1444, 1521, 1600, 1681, 1764, 1849, 1936, 2025, 2116,
 2209, 2304, 2401, 2500, ...]


上の例で colab 環境で非同期処理が問題なく動くことが確認できたみたい。

---
次のもネットで紹介されていた例で、ハローワールド並行処理版


In [ ]:
%%writefile temp.exs
parent = self()
 
spawn_link(fn ->
  send parent, {:msg, "hello world"}
end)
 
receive do
  {:msg, contents} -> IO.puts contents
end

Overwriting temp.exs


In [ ]:
!elixir temp.exs

hello world


上の例でやっていることはつぎのような流れである。

1. spawn_linkという関数に渡された関数が、関数の内容を実行する。
2. 新しく作られたプロセス側では、メインプロセス側（parent）に “hello world” というメッセージを送る。
3. メインプロセス側は、どこからかメッセージが来ないかを待ち受けて（receive）、メッセージが来たらそれをコンソールに表示する。

In [ ]:
# 実験 とりあえず理解しない。 colab 環境でどうかだけ調べる。
%%writefile chain.exs
defmodule Chain do
  def counter(next_pid) do
    receive do
      n -> send next_pid, n + 1
    end
  end
 
  def create_processes(n) do
    last = Enum.reduce 1..n, self(),
      fn (_, send_to) -> spawn(Chain, :counter, [send_to]) end
    send last, 0
    receive do
      final_answer when is_integer(final_answer) ->
        "Result is #{inspect(final_answer)}"
    end
  end
 
  def run(n) do
    IO.puts inspect :timer.tc(Chain, :create_processes, [n])
  end
end

Writing chain.exs


In [ ]:
!elixir --erl "+P 1000000" -r chain.exs  -e "Chain.run(1_000_000)"

{4638957, "Result is 1000000"}


記事 https://ubiteku.oinker.me/2015/12/22/elixir試飲-2-カルチャーショックに戸惑う-並行指向プ/ のマシン Macbook Pro – 3 GHz Intel Core i7, 16GB RAM では 7 秒のところ、colab では 5 秒で終わってるね!!!!

手元のwindowsマシン intel core i5-9400 8gb ram でやったら次のようになった。  
{3492935, "Result is 1000000"}

あれ、速いじゃん!!!!

---
コメントは `#`

In [ ]:
%%writefile temp.exs
# コメント実験
str = "helloworld!!!!"
IO.puts str

Overwriting temp.exs


In [ ]:
!elixir temp.exs

helloworld!!!!


---
n 進数、整数 integer

In [ ]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!elixir -e 'IO.puts 1000_000_00_0'

15
4095
65535
1000000000


整数型に上限下限 fixed limit はない。 factorial(10000) が計算できる。今はしない。


---
問題

10進数を $n$ 進数にベースを変えるのはどうするか。 python では `int()`, `bin()`, `oct()`, `hex()` があった。

In [ ]:
# python
print(0b1111)
print(0o7777)
print(0xffff)
print(int('7777',8))
print(bin(15))
print(oct(4095))
print(hex(65535))

15
4095
65535
4095
0b1111
0o7777
0xffff


In [ ]:
!elixir -e 'IO.puts 0b1111'
!elixir -e 'IO.puts 0o7777'
!elixir -e 'IO.puts 0xffff'
!echo
# Integer.to_string() と言う関数を使う
# <> はバイナリー連結
!elixir -e 'IO.puts "0b" <> Integer.to_string(15,2)'
!elixir -e 'IO.puts "0o" <> Integer.to_string(4095,8)'
!elixir -e 'IO.puts "0x" <> Integer.to_string(65535,16)'

15
4095
65535

0b1111
0o7777
0xFFFF


浮動小数点数 floating-point number

In [ ]:
!elixir -e 'IO.puts 1.532e-4'
# .0 とか 1. とかはエラーになる
!elixir -e 'IO.puts 98099098.0809898888'
!elixir -e 'IO.puts 0.00000000000000000000000001' #=> 1.0e-26
!elixir -e 'IO.puts 90000000000000000000000000000000000000000000000000000000'

1.532e-4
98099098.08098988
1.0e-26
999999999999999999999999999999999999999
90000000000000000000000000000000000000000000000000000000


文字列 string

string という型はない、みたい。

---
質問 型を調べる関数はあるか。type() とか。


In [ ]:
!elixir -e 'IO.puts "日本語が書けますか"'
!elixir -e 'IO.puts "日本語が書けます"'

日本語が書けますか
日本語が書けます


In [ ]:
# 関数に括弧をつけることができる
# \ で escape できる
!elixir -e 'IO.puts (0b1111)'
!elixir -e 'IO.puts ("にほんご\n日本語")'
!elixir -e "IO.puts ('にほんご\n\"日本語\"')"

15
にほんご
日本語
にほんご
"日本語"


In [ ]:
# 文字連結 `+` ではない!!!!
!elixir -e 'IO.puts("ABCD"<>"EFGH")'

ABCDEFGH


`<>` と言う記号はバイナリ連結ということらしい。

---
値の埋め込み

`#{変数名}` を記述することで、変数の値を埋め込むことができる。


In [ ]:
!elixir -e 'val = 1000; IO.puts "val = #{val}"'

val = 1000


---
真偽値

elixir の 真偽値は true と false (小文字) で false と nil が false でそれ以外は true



In [ ]:
!elixir -e 'if true do IO.puts "true" end'
!elixir -e 'if True do IO.puts "true" end'
!elixir -e 'if False do IO.puts "true" end' # False が大文字なので
!elixir -e 'if false do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if nil do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if 0 do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if (-1) do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if [] do IO.puts "true" else IO.puts "false" end'
!elixir -e 'if "" do IO.puts "true" else IO.puts "false" end'

true
true
true
false
false
true
true
true
true


`null` はない。

---
**マッチ演算子 `=`**

マッチ演算子 `=` はマッチ演算子である。 マッチ演算子を通して値を代入し、その後、マッチさせることができる。マッチすると、方程式の結果が返され、失敗すると、エラーになる。

In [ ]:
!elixir -e 'IO.puts a = 1'
!elixir -e 'a =1; IO.puts 1 = a'
!elixir -e 'a =1; IO.puts 2 = a'

1
1
** (MatchError) no match of right hand side value: 1
    (stdlib 3.17) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.17) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.17) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.13.0) lib/code.ex:404: Code.validated_eval_string/3


In [ ]:
!elixir -e 'IO.inspect a = [1,2,3]' # リストは puts で表示できないので inspect を使う
!elixir -e '[a,b,c] = [1,2,3]; IO.puts c; IO.puts b'


[1, 2, 3]
3
2


上の例は、elixir は マッチ演算子 `=` があると左右がマッチするように最善を尽くす。 そのため、`[a,b,c] = [1,2,3]` で a,b,c に値が代入される。

In [ ]:
!elixir -e 'IO.inspect [1,2,[3,4,5]]'
!elixir -e '[a,b,c] = [1,2,[3,4,5]]; IO.inspect c; IO.inspect b'

[1, 2, [3, 4, 5]]
[3, 4, 5]
2


In [ ]:
# 実験 => エラー
!elixir -e 'IO.insepct [a,b] = [1,2,3]'

** (MatchError) no match of right hand side value: [1, 2, 3]
    (stdlib 3.15) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.15) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.15) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.12.0) lib/code.ex:656: Code.eval_string_with_error_handling/3


In [ ]:
# 実験
!elixir -e 'IO.inspect a = [[1,2,3]]'
!elixir -e 'IO.inspect [a] = [[1,2,3]]'
!elixir -e '[a] = [[1,2,3]]; IO.inspect a'

[[1, 2, 3]]
[[1, 2, 3]]
[1, 2, 3]


In [ ]:
# 実験 => エラー
!elixir -e 'IO.insepct [a,b] = [a,b]'

  nofile:1

** (CompileError) nofile:1: undefined function a/0
    (stdlib 3.15) lists.erl:1358: :lists.mapfoldl/3


In [ ]:
# 実験 アトムについては後述
!elixir -e 'IO.puts a = :a'
!elixir -e 'a = :a; IO.inspect a = a'
!elixir -e 'a = :a; IO.puts a = a'
!elixir -e 'IO.puts :b'

a
:a
a
b


アンダースコア `_` で値を無視する。 ワルドカード。

なんでも受け付ける。

In [ ]:
!elixir -e 'IO.inspect [1,_,_]=[1,2,3]'

[1, 2, 3]


In [ ]:
!elixir -e 'IO.inspect [1,_,_]=[1,"cat","dog"]'

[1, "cat", "dog"]


変数は、バインド (束縛、紐付け) されると変更できない。

かと思ったらできてしまう。


In [ ]:
!elixir -e 'a = 1; IO.puts a = 2'

2


元の変数を指し示すピン演算子 (`^` カレット) がある。

In [ ]:
!elixir -e 'a = 1; IO.puts ^a = 2'

** (MatchError) no match of right hand side value: 2
    (stdlib 3.15) erl_eval.erl:450: :erl_eval.expr/5
    (stdlib 3.15) erl_eval.erl:893: :erl_eval.expr_list/6
    (stdlib 3.15) erl_eval.erl:408: :erl_eval.expr/5
    (elixir 1.12.0) lib/code.ex:656: Code.eval_string_with_error_handling/3


メモ $\quad$ 普通の関数型言語のように変数は変更できないルールにしてしまった方が簡単ではなかったか、と思わないでもない。 変数を不変にする、const 宣言みたいなのはないのか。

リストは不変 immutable なので安心。



In [ ]:
# 大文字にする capitalize
!elixir -e 'IO.puts name = String.capitalize "elixir"'

Elixir


In [ ]:
# 大文字にする upcase
!elixir -e 'IO.puts String.upcase "elixir"'

ELIXIR


# アトム

アトムは名前がそのまま値となる定数である。

**名前の前にコロン `:` をつけることでアトムになる。**

アトムの名前は utf-8 文字列 (記号を含む)、数字、アンダースコア `_` 、`@` で、終端文字としてのみ「!」や「?」が使える。

:fred $\quad$ :is_binary? $\quad$ ​:var@2 $\quad$ ​:<> $\quad$ ​:=== 

​:"func/3"​ $\quad$ ​​:"long john silver" $\quad$ ​:эликсир

​:mötley_crüe

メモ

In [ ]:
# 実験 アトムは宣言しないで突然使える
!elixir -e 'IO.puts :fred'

fred


In [ ]:
# 実験
!elixir -e 'IO.puts true === :true'
!elixir -e 'IO.puts :true'
!elixir -e 'IO.puts false === :false'

true
true
true


In [ ]:
# 実験 
!elixir -e 'IO.puts :fred'
!elixir -e 'IO.puts :is_binary?'
!elixir -e 'IO.puts :var@2'
!elixir -e 'IO.puts :<>'
!elixir -e 'IO.puts :==='
# セミコロンを含むアトムは iex 上では使えるが、シェルワンライナーでは使えない  
# unexpected token: "​" と言うエラーになる
# colab の環境だけでなく、通常のシェルでも同じ
# ファイルにしたプログラムでは使えるので問題ない
# !elixir -e 'IO.puts :"func/3"​' 
# !elixir -e 'IO.puts ​​:"long john silver"'
!elixir -e 'IO.puts :эликсир'
!elixir -e 'IO.puts :mötley_crüe'
!elixir -e 'IO.puts :日本語はどうか'

fred
is_binary?
var@2
<>
===
эликсир
mötley_crüe
日本語はどうか


演算子

In [ ]:
!elixir -e 'IO.puts 1 + 2'
!elixir -e 'x = 10; IO.puts x + 1'
!elixir -e 'IO.puts 1 - 2'
!elixir -e 'x = 10; IO.puts x - 1'
!elixir -e 'IO.puts 5 * 2'
!elixir -e 'x = 10; IO.puts x * 4'
!echo 
!elixir -e 'IO.puts 5 / 2'
!elixir -e 'x = 10; IO.puts x / 3'

3
11
-1
9
10
40

2.5
3.3333333333333335


In [ ]:
# 浮動少数ではなく整数としての結果がほしい場合は div 関数を使用
!elixir -e 'IO.puts div(10,5)'
!elixir -e 'IO.puts div(10,4)'
# 割り算の余り、剰余を求める場合は rem関数を使用
!elixir -e 'IO.puts rem(10,4)'
!elixir -e 'IO.puts rem(10,3)'
!elixir -e 'IO.puts rem(10,2)'

2
2
2
1
0


In [ ]:
# 比較演算子
!elixir -e 'IO.puts 1 == 1'
!elixir -e 'IO.puts 1 != 1'
!elixir -e 'IO.puts ! (1 != 1)'
!echo
!elixir -e 'IO.puts 20.0 == 20'
!elixir -e 'IO.puts 20.0 === 20'
!elixir -e 'IO.puts 20.0 !== 20'

true
false
true

true
false
true


In [ ]:
# 論理演算子
# 論理和
!elixir -e 'IO.puts "ABC" == "ABC" || 20 == 30'
!elixir -e 'IO.puts "ABC" == "abc" || 20 == 30'
!echo
# 論理積
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 20'
!elixir -e 'IO.puts "ABC" == "ABC" && 20 == 30'
!elixir -e 'IO.puts "ABC" == "def" && 10 > 100'
!echo
# 否定
!elixir -e 'IO.puts !("ABC" == "ABC")'
!elixir -e 'IO.puts !("ABC" == "DEF")'

true
false

true
false
false

false
true


range

メモ $\quad$ range は型ではなく、struct である。 構造体?

`start..end` で表現される、とあるが、1..10 と書けばそれで range なのか?


In [ ]:
!elixir -e 'IO.inspect Enum.to_list(1..3)'
!elixir -e 'IO.inspect Enum.to_list(0..10//3)'
!elixir -e 'IO.inspect Enum.to_list(0..10//-3)'
!elixir -e 'IO.inspect Enum.to_list(10..0//-3)'
!elixir -e 'IO.inspect Enum.to_list(1..1)'
!elixir -e 'IO.inspect Enum.to_list(1..-1)'
!elixir -e 'IO.inspect Enum.to_list(1..1//2)'
!elixir -e 'IO.inspect Enum.to_list(1..-1//2)'
!elixir -e 'IO.inspect Enum.to_list(1..-1//-2)'

[1, 2, 3]
[0, 3, 6, 9]
[]
[10, 7, 4, 1]
[1]
[1, 0, -1]
[1]
[]
[1, -1]


In [ ]:
!elixir -e 'IO.inspect 1..9//2'

1..9//2


正規表現 regular expression

正規表現も型ではなく、struct である。

In [ ]:
!elixir -e 'IO.inspect Regex.run ~r{[aiueo]},"catapillar"'

["a"]


In [ ]:
!elixir -e 'IO.inspect Regex.scan ~r{[aiueo]},"catapillar"'

[["a"], ["a"], ["i"], ["a"]]


In [ ]:
!elixir -e 'IO.inspect Regex.split ~r{[aiueo]},"catapillar"'

["c", "t", "p", "ll", "r"]


In [ ]:
!elixir -e 'IO.inspect Regex.replace ~r{[aiueo]},"catapillar", "*"'

"c*t*p*ll*r"


# コレクション型

## タプル

タプルは波括弧 brace を用いて定義する。

タプルに限らず elixir のコレクションはすべて要素のタイプを限定しない。

通常 2 から 4 の要素であり、それ以上の要素数の場合、map や struct の利用を考える。

タプルは関数の返り値に便利に利用される。

パターンマッチングと組み合わせて使われる。

---
cf. タプル以外の波括弧 brace の使用
* 値の代入`#{変数名}` 
* 正規表現 Regex `r{}` 
* マップ `%{}` 

In [ ]:
!elixir -e 'IO.inspect {3.14, :pie, "Apple"}'

{3.14, :pie, "Apple"}


In [ ]:
!elixir -e '{status, count, action} = {3.14, :pie, "next"}; IO.puts action'

next


In [ ]:
# 実験
# タプルの使い方の例 
!echo hello > temp.txt
!elixir -e '{status, file} = File.open("temp.txt"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.read("temp.txt"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.read("temp02.txt"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.write("temp.txt", "goodbye"); IO.inspect {status, file}'
!elixir -e '{status, file} = File.read("temp.txt"); IO.inspect {status, file}'


In [ ]:
# 実験 タプルに ++ は使えるか。 => 使えない <> も使えない
# !elixir -e 'IO.inspect {3.14, :pie, "Apple"} ++ {3}'
# 実験 タプルに head は使えるか。 => 使えない
# !elixir -e 'IO.inspect hd {3.14, :pie, "Apple"}'
# 実験 タプルにパターンマッチングは使えるか。 => 使える
!elixir -e '{a,b,c} =  {3.14, :pie, "Apple"}; IO.inspect [c,a,b]'

["Apple", 3.14, :pie]


In [ ]:
# 実験
# 項目の入れ替え
!elixir -e 'a=1; b=3; {b,a}={a,b}; IO.inspect {a,b}'
!elixir -e 'a=1; b=3; c=5; d= 7; {d,c,b,a}={a,b,c,d}; IO.inspect {a,b,c,d}'

{3, 1}
{7, 5, 3, 1}


In [ ]:
# 実験
# タプルの要素にタプルはあるか
!elixir -e 'IO.inspect {3.14, :pie, "Apple", {3}}'

{3.14, :pie, "Apple", {3}}


## リスト

他の言語の配列 array と elixir のリストは違うので注意。 lisp のリストと似たような概念である。

カラのリストでなければ、head (hd) と tail (tl) がある。hd は頭の1つで tl はそれ以降全部。

In [ ]:
# リスト
!elixir -e 'IO.inspect [3.14, :pie, "Apple"]'
!elixir -e 'IO.inspect hd [3.14]'
!elixir -e 'IO.inspect tl [3.14]'

[3.14, :pie, "Apple"]
3.14
[]


In [ ]:
# リスト先頭への追加(高速)
!elixir -e 'IO.inspect ["π" | [3.14, :pie, "Apple"]]'
# リスト末尾への追加(低速)
!elixir -e 'IO.inspect [3.14, :pie, "Apple"] ++ ["Cherry"]'

["π", 3.14, :pie, "Apple"]
[3.14, :pie, "Apple", "Cherry"]


上と下のコードセルでリストの連結を行っているが、++/2 演算子を用いている。 この `++/2` という表記は `++` が演算子自体で `/2` がアリティ (引数の数) を表す。 



---
質問 $\quad$ アリティとはなにか。

---
質問 $\quad$ リストの連結に `++` で文字列の連結 `<>` なのはなぜか。 オーバーライディングはあるのか。 文字列 string はリストではないのか。 長さを測る関数も別々なのか。

In [ ]:
# リストの連結
!elixir -e 'IO.inspect  [1, 2] ++ [3, 4, 1]'

[1, 2, 3, 4, 1]


In [ ]:
# リストの減算 
#  --/2 演算子は存在しない値を引いてしまってもオッケー
!elixir -e 'IO.inspect ["foo", :bar, 42] -- [42, "bar"]'
# 重複した値の場合、右辺の要素のそれぞれに対し、左辺の要素のうち初めて登場した同じ値が順次削除
!elixir -e 'IO.inspect [1,2,2,3,2,3] -- [1,2,3,2]'
# リストの減算の値のマッチには strict comparison が使われている
!elixir -e 'IO.inspect  [2] -- [2.0]'
!elixir -e 'IO.inspect  [2.0] -- [2.0]'

["foo", :bar]
[2, 3]
[2]
[]


In [ ]:
# head /tail
!elixir -e 'IO.inspect hd [3.14, :pie, "Apple"]'
!elixir -e 'IO.inspect tl [3.14, :pie, "Apple"]'

3.14
[:pie, "Apple"]


---
リストを頭部と尾部に分けるのに

* パターンマッチング
* cons 演算子( `|` )

を使うこともできる。

In [ ]:
!elixir -e '[head | tail] = [3.14, :pie, "Apple"]; IO.inspect head; IO.inspect tail'

3.14
[:pie, "Apple"]


## キーワードリスト

キーワードリストとマップは elixir の連想配列である。

キーワードリストは最初の要素がアトムのタプルからなる特別なリストで、リストと同様の性能になる。

In [ ]:
# キーワードリスト
!elixir -e 'IO.inspect [foo: "bar", hello: "world"]'
# タプルのリストとしても同じ
!elixir -e 'IO.inspect [{:foo, "bar"}, {:hello, "world"}]'
!elixir -e 'IO.inspect [foo: "bar", hello: "world"] == [{:foo, "bar"}, {:hello, "world"}]'

[foo: "bar", hello: "world"]
[foo: "bar", hello: "world"]
true


キーワードリストの 3 つの特徴

* キーはアトムである。
* キーは順序付けされている。
* キーの一意性は保証されない。

こうした理由から、キーワードリストは関数にオプションを渡すためによく用いられる。


In [ ]:
# 実験 リストの角括弧は省略できる
!elixir -e 'IO.inspect foo: "bar", hello: "world"'

[foo: "bar", hello: "world"]


In [ ]:
# 実験
!elixir -e 'IO.inspect [1, fred: 1, dave: 2]'
!elixir -e 'IO.inspect {1, fred: 1, dave: 2}'
!elixir -e 'IO.inspect {1, [{:fred,1},{:dave, 2}]}'

[1, {:fred, 1}, {:dave, 2}]
{1, [fred: 1, dave: 2]}
{1, [fred: 1, dave: 2]}


## マップ

* キーワードリストとは違ってどんな型のキーも使える。
* 順序付けされない。
* キーの一意性が保証されている。重複したキーが追加された場合は、前の値が置き換えられる。
* 変数をマップのキーにできる。
* `%{}` 構文で定義する。



In [ ]:
!elixir -e 'IO.inspect %{:foo => "bar", "hello" => :world}'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map[:foo]'
!elixir -e 'map = %{:foo => "bar", "hello" => :world}; IO.inspect map["hello"]'
!echo
!elixir -e 'key = "hello"; IO.inspect %{key => "world"}'
!echo
!elixir -e 'IO.inspect %{:foo => "bar", :foo => "hello world"}'

%{:foo => "bar", "hello" => :world}
"bar"
:world

%{"hello" => "world"}

  nofile:1

%{foo: "hello world"}


アトムのキーだけを含んだマップには特別な構文がある。

In [ ]:
!elixir -e 'IO.inspect %{foo: "bar", hello: "world"} == %{:foo => "bar", :hello => "world"}'

true


In [ ]:
# 加えて、アトムのキーにアクセスするための特別な構文がある。
!elixir -e 'map = %{foo: "bar", hello: "world"}; IO.inspect map.hello'
!elixir -e 'map = %{foo: "bar", hello: "world"}; IO.inspect map[:hello]'
!elixir -e 'map = %{:foo => "bar", :hello =>  "world"}; IO.inspect map[:hello]'

"world"
"world"
"world"


---
質問 map の特別な構文

1. `=>` の代わりにコロン `:` を使う
2. 要素を取り出すのに `[]` の代わりにピリオド `.` を使う

は不要ではないか。不要だが見かけが良くなる、ということか。普通はどっちを使うのか。無駄に構文を複雑にするだけのような気がする。

多分まず Python の dict でコロン `:` を使うこと、Ruby は `=>` を使うが糖衣構文としてコロン `:` が使えてその形が主流であることから、見かけ大切ということでこうなったのではないか。キーにアトムを使うことが前提ならば生産性が上がるかもしれない。キーであることを示すコロンが不要になる。fat arrow よりコロンの方が短い。map の定義が同時に行われる。要素の取り出しピリオドを使う点についても同様。ということは基本的にこの構文になる、と言う事だろう。

In [ ]:
# マップの更新のための構文がある (新しい map が作成される)
# この構文は、マップに既に存在するキーを更新する場合にのみ機能する
!elixir -e 'map = %{foo: "bar", hello: "world"}; IO.inspect %{map | foo: "baz"}'

%{foo: "baz", hello: "world"}


In [ ]:
# 新しいキーを作成するには、`Map.put/3` を使用
!elixir -e 'map = %{hello: "world"}; IO.inspect  Map.put(map, :foo, "baz")'

%{foo: "baz", hello: "world"}


---
質問 binary については良くわからないので別途。

# バイナリ binary

In [ ]:
# binaries 
!elixir -e 'IO.inspect <<1,2>>'
!elixir -e 'IO.inspect <<1,10>>'
!elixir -e 'bin = <<1,10>>; IO.inspect byte_size bin'
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect bin'
!elixir -e 'IO.puts Integer.to_string(213,2)'
!elixir -e 'IO.puts 0b11'
!elixir -e 'IO.puts 0b0101'
!echo
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect byte_size bin'
!elixir -e 'bin = <<3::size(2),5::size(4),1::size(2)>>; IO.inspect :io.format("~-8.2b~n",:binary.bin_to_list(bin))'

<<1, 2>>
<<1, 10>>
2
<<213>>
11010101
3
5

1
11010101
:ok


----
** Date and Time 日付 **

In [ ]:
# Date and Time
!elixir -e 'IO.inspect Date.new(2021,6,2)'

{:ok, ~D[2021-06-02]}


In [ ]:
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect d1'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect Date.day_of_week(d1)'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect Date.add(d1,7)'
!elixir -e '{:ok, d1}=Date.new(2021,6,2); IO.inspect d1, structs: false'

~D[2021-06-02]
3
~D[2021-06-09]
%{__struct__: Date, calendar: Calendar.ISO, day: 2, month: 6, year: 2021}


`~D[...]` や `~T[...]` は elixir の シギル sigil である。 文字列とバイナリーのところで説明する。

# help について
メモ $\quad$ 関数の調べ方

Helper の使い方。 help, type, info, information とか。

下のコードセルにあるように、対象のモジュールの関数名を調べ、そのヘルプを見ればけっこうくわしくわかる。

コメントアウトしてあるのは出力が大きいので、とりあえずコメントアウトして出力を抑制してある。

具体的には、Enum にあたるところにモジュール名を入れて関数のリストを出す。 Ctrl+A Ctrl+C でコピーして vscode などでペーストして読む。 調べたい関数名をヘルプの、Enum.all?/1 のところに入れて出力をコピーして、vscode などでペーストして読む


In [ ]:
# !elixir -e 'Enum.__info__(:functions) |> Enum.each(fn({function, arity}) -> IO.puts "#{function}/#{arity}" end)'

In [ ]:
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h Enum.all?/1'

In [ ]:
# h 単独のドキュメントを見たい
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h'

In [ ]:
# i というのもある
# !elixir -e 'x = [3,2]; require IEx.Helpers;IEx.Helpers.i x'

In [ ]:
# !elixir -e 'require IEx.Helpers;IEx.Helpers.h IO'

# Enum モジュール

Enum はリストなどコレクションを列挙するための一連のアルゴリズム。

* all?、any?
* chunk_every、chunk_by、map_every
* each
* map、filter、reduce
* min、max
* sort、uniq、uniq_by
* キャプチャ演算子 `(&)`


In [ ]:
# all? 関数を引数で受け取り、リストの全体が true の時、true を返す
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) == 3 end)'
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) >1 end)'

false
true


In [ ]:
# any? 少なくとも1つの要素が true と評価された場合に true を返す
!elixir -e 'IO.puts Enum.any?(["foo", "bar", "hello"], fn(s) -> String.length(s) == 5 end)'

true


In [ ]:
# chunk_every リストを小さなグループに分割する
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 2)'
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 3)'
!elixir -e 'IO.inspect Enum.chunk([1, 2, 3, 4, 5, 6], 4)'

[[1, 2], [3, 4], [5, 6]]
[[1, 2, 3], [4, 5, 6]]
[[1, 2, 3, 4]]


In [ ]:
# chunk_by 関数の戻り値が変化することによって分割する
!elixir -e 'IO.inspect Enum.chunk_by(["one", "two", "three", "four", "five"], fn(x) -> String.length(x) end)'
!elixir -e 'IO.inspect Enum.chunk_by(["one", "two", "three", "four", "five", "six"], fn(x) -> String.length(x) end)'

[["one", "two"], ["three"], ["four", "five"]]
[["one", "two"], ["three"], ["four", "five"], ["six"]]


In [ ]:
# map_every nth ごとに map 処理する
!elixir -e 'IO.inspect Enum.map_every(1..10, 3, fn x -> x + 1000 end)'
!elixir -e 'IO.inspect Enum.map_every(1..10, 1, fn x -> x + 1000 end)'
!elixir -e 'IO.inspect Enum.map_every(1..10, 0, fn x -> x + 1000 end)'

[1001, 2, 3, 1004, 5, 6, 1007, 8, 9, 1010]
[1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [ ]:
# each 新しい値を生成することなく反復する。返り値は:ok というアトム。
!elixir -e 'IO.inspect Enum.each(["one", "two", "three"], fn(s) -> IO.puts(s) end)'
!elixir -e 'IO.puts Enum.each(["one", "two", "three"], fn(s) -> IO.puts(s) end)'

one
two
three
:ok


In [ ]:
# map 関数を各要素に適用して新しいリストを生み出す
!elixir -e 'IO.inspect Enum.map([0, 1, 2, 3], fn(x) -> x - 1 end)'

[-1, 0, 1, 2]


In [ ]:
# min 最小の値を探す。 リストが空の場合エラーになる
# リストが空だったときのために予め最衣装血を生成する関数を渡すことができる
!elixir -e 'IO.inspect Enum.min([5, 3, 0, -1])'
!elixir -e 'IO.inspect Enum.min([], fn -> :foo end)'

-1
:foo


In [ ]:
# max 最大の(max/1)値を返す
!elixir -e 'IO.inspect Enum.max([5, 3, 0, -1])'
!elixir -e 'IO.inspect Enum.max([], fn -> :bar end)'

5
:bar


In [ ]:
# filter 与えられた関数によって true と評価された要素だけを得る
!elixir -e 'IO.inspect Enum.filter([1, 2, 3, 4], fn(x) -> rem(x, 2) == 0 end)'
!elixir -e 'IO.inspect Enum.filter([], fn(x) -> rem(x, 2) == 0 end)'

[2, 4]
[]


In [ ]:
# reduce リストを関数に従って単一の値へ抽出する。 accumulator を指定できる。
# accumulator が与えられない場合は最初の要素が用いられる。
!elixir -e 'IO.inspect Enum.reduce([1, 2, 3], 10, fn(x, acc) -> x + acc end)'
!elixir -e 'IO.inspect Enum.reduce([1, 2, 3], fn(x, acc) -> x + acc end)'
!elixir -e 'IO.inspect Enum.reduce(["a","b","c"], "1", fn(x,acc)-> x <> acc end)'

16
6
"cba1"


In [ ]:
# sort `sort/1` はソートの順序に Erlangの Term 優先順位 を使う
!elixir -e 'IO.inspect Enum.sort([5, 6, 1, 3, -1, 4])'
!elixir -e 'IO.inspect Enum.sort([:foo, "bar", Enum, -1, 4])'

# `sort/2` は、順序を決める為の関数を渡すことができる
!elixir -e 'IO.inspect Enum.sort([%{:val => 4}, %{:val => 1}], fn(x, y) -> x[:val] > y[:val] end)'

# なしの場合
!elixir -e 'IO.inspect Enum.sort([%{:count => 4}, %{:count => 1}])'

# sort/2 に :asc または :desc をソート関数として渡すことができる
!elixir -e 'IO.inspect Enum.sort([2, 3, 1], :desc)'


[-1, 1, 3, 4, 5, 6]
[-1, 4, Enum, :foo, "bar"]
[%{val: 4}, %{val: 1}]
[%{count: 1}, %{count: 4}]
[3, 2, 1]


In [ ]:
# uniq 重複した要素を取り除く
!elixir -e 'IO.inspect Enum.uniq([1, 2, 3, 2, 1, 1, 1, 1, 1])'
[1, 2, 3]

# uniq_by 重複した要素を削除するが、ユニークかどうか比較を行う関数を渡せる
!elixir -e 'IO.inspect Enum.uniq_by([%{x: 1, y: 1}, %{x: 2, y: 1}, %{x: 3, y: 3}], fn coord -> coord.y end)'

[1, 2, 3]
[%{x: 1, y: 1}, %{x: 3, y: 3}]


# キャプチャ演算子 `&` を使用した Enum と無名関数 

elixir の Enum モジュール内の多くの関数は、引数として無名関数を取る。

これらの無名関数は、多くの場合、キャプチャ演算子 `&` を使用して省略形で記述される。



In [ ]:
# 無名関数でのキャプチャ演算子の使用 
!elixir -e 'IO.inspect Enum.map([1,2,3], fn number -> number + 3 end)'
!elixir -e 'IO.inspect Enum.map([1,2,3], &(&1 + 3))'
!elixir -e 'plus_three = &(&1 + 3);IO.inspect Enum.map([1,2,3], plus_three)'

[4, 5, 6]
[4, 5, 6]
[4, 5, 6]


In [ ]:
# Enum.all? でもキャプチャ演算子が使えるか
# all? 関数を引数で受け取り、リストの全体が true の時、true を返す
# !elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) == 3 end)'
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], &(String.length(&1)==3))'
# !elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], fn(s) -> String.length(s) >1 end)'
!elixir -e 'IO.puts Enum.all?(["foo", "bar", "hello"], &(String.length(&1)>1))'

false
true


---
# パターンマッチング

パターンマッチングでは、値、データ構造、関数をマッチすることができる。

* マッチ演算子
* ピン演算子

In [ ]:
# マッチ演算子 `=` はマッチ演算子である。 マッチ演算子を通して値を代入し、
# その後、マッチさせることができる。マッチすると、方程式の結果が返され、
# 失敗すると、エラーになる
!elixir -e 'IO.puts x = 1'
!elixir -e 'x = 1;IO.puts 1 = x'
# !elixir -e 'x = 1;IO.puts 2 = x'
#=> (MatchError) no match of right hand side value: 1

1
1


In [ ]:
# リストでのマッチ演算子
!elixir -e 'IO.inspect list = [1, 2, 3]'
!elixir -e 'list = [1, 2, 3]; IO.inspect [1, 2, 3] = list'
# !elixir -e 'list = [1, 2, 3]; IO.inspect [] = list'
#=> (MatchError) no match of right hand side value: [1, 2, 3]

[1, 2, 3]
[1, 2, 3]


In [ ]:
!elixir -e 'list = [1, 2, 3]; IO.inspect [1 | tail] = list'
!elixir -e 'list = [1, 2, 3];  [1 | tail] = list; IO.inspect tail'

[1, 2, 3]
[2, 3]


In [ ]:
# タプルとマッチ演算子
!elixir -e 'IO.inspect {:ok, value} = {:ok, "Successful!"}'
!elixir -e '{:ok, value} = {:ok, "Successful!"}; IO.inspect  value'

{:ok, "Successful!"}
"Successful!"


---
**ピン演算子**

マッチ演算子は左辺に変数が含まれている時に代入操作を行う。 

この変数を再び束縛するという挙動は望ましくない場合がある。 そうした状況のために、ピン演算子 `^` がある。

ピン演算子で変数を固定すると、新しく再束縛するのではなく既存の値とマッチする。


In [ ]:
# ピン演算子
!elixir -e 'IO.inspect x = 1'
# !elixir -e 'x = 1; IO.inspect ^x = 2'
#=> ** (MatchError) no match of right hand side value: 2
!elixir -e 'x = 1; IO.inspect {x, ^x} = {2, 1}'
!elixir -e 'x = 1;{x, ^x} = {2, 1}; IO.inspect x'
!echo
!elixir -e 'IO.inspect key = "hello"'
!elixir -e 'key = "hello"; IO.inspect %{^key => value} = %{"hello" => "world"}'
!elixir -e 'key = "hello"; %{^key => value} = %{"hello" => "world"}; IO.inspect value'
!elixir -e 'key = "hello"; %{^key => value} = %{"hello" => "world"}; IO.inspect value'

1
{2, 1}
2

"hello"
%{"hello" => "world"}
"world"
"world"


In [ ]:
# 関数の clause でのピン演算子
!elixir -e 'IO.inspect greeting = "Hello"'
!elixir -e 'greeting = "Hello"; IO.inspect greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "#{greeting},#{name}" end'
!elixir -e 'greeting = "Hello"; greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "#{greeting},#{name}" end; IO.inspect greet.("Hello","Sean")'
!elixir -e 'greeting = "Hello"; greet = fn (^greeting, name) -> "Hi #{name}"; (greeting, name) -> "#{greeting},#{name}" end; IO.inspect greet.("Mornin","Sean")'

"Hello"
#Function<43.65746770/2 in :erl_eval.expr/5>
"Hi Sean"
"Mornin,Sean"


# 制御構造 control structure

* if と unless
* case
* cond
* with

if と unless 

elixir の if と unless は ruby と同じ。

elixir は if と unless はマクロとして定義されている。

この実装は kernel module で知ることができる。

elixir では偽とみなされる値は nil と真理値の false だけだということに留意。

In [ ]:
%%writefile temp.exs
IO.puts (
    if String.valid?("Hello") do
        "Valid string!"
    else
        "Invalid string."
    end)

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Valid string!


In [ ]:
%%writefile temp.exs
if "a string value" do
  IO.puts "Truthy"
end

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Truthy


In [ ]:
# unless/2 は if/2 の逆で、条件が否定される時だけ作用する
%%writefile temp.exs
unless is_integer("hello") do
  IO.puts "Not an Int"
end

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Not an Int


In [ ]:
# 実験 シェルワンライナー版 do や end の前後にセミコロンは要らない
!elixir -e 'unless is_integer("hello") do IO.puts "Not an Int" end'

Not an Int


In [ ]:
# 複数のパターンにマッチする場合、case/2 を使う
%%writefile temp.exs
IO.puts(
    case {:error, "Hello World"} do
        {:ok, result} -> result
        {:error, _} -> "Uh oh!"
        _ -> "Catch all"
    end   
)

In [ ]:
!elixir temp.exs

Uh oh!


In [ ]:
# アンダースコア _ 変数は case/2 命令文の中に含まれる重要な要素
# これが無いと、マッチするものが見あたらない場合にエラーが発生する
# エラーの例
!elixir -e 'case :even do :odd -> IO.puts "Odd" end'

** (CaseClauseError) no case clause matching: :even
    (stdlib 3.17) erl_eval.erl:973: :erl_eval.case_clauses/6
    (elixir 1.13.0) lib/code.ex:404: Code.validated_eval_string/3


In [ ]:
# アンダースコア _ を"他の全て"にマッチする else と考えること
!elixir -e 'case :even do :odd -> IO.puts "Odd"; _ -> IO.puts "Not odd" end'

Not Odd


In [ ]:
# case/2 はパターンマッチングに依存しているため、パターンマッチングと同じルールや制限が全て適用される
# 既存の変数に対してマッチさせようという場合にはピン ^ 演算子を使う
!elixir -e 'pie=3.14; IO.puts(case "cherry pie" do ^pie -> "Not so tasty"; pie -> "I bet #{pie} is tasty" end)'
!elixir -e 'pie=3.14; IO.puts(case "cherry pie" do pie -> "Not so tasty"; pie -> "I bet #{pie} is tasty" end)'

I bet cherry pie is tasty
  nofile:1

Not so tasty


In [ ]:
# case/2 はガード節に対応している
# 公式ドキュメントの Expressions allowed in guard clauses を参照
!elixir -e 'IO.puts(case {1, 2, 3} do {1, x, 3} when x > 0 ->  "Will match"; _ -> "Wont match" end)'

Will match


---
ガード節とは何か?

公式ドキュメントの Expressions allowed in guard clauses を参照

In [ ]:
# cond
!elixir -e 'IO.puts (cond do 2+2==5 -> "This will not be true"; 2*2==3 -> "Nor this"; 1+1 == 2 -> "But this will" end)'

But this will


In [ ]:
# cond も case と同様マッチしない場合にエラーになるので、true になる条件を定義する
!elixir -e 'IO.puts (cond do 7+1==0 -> "Incorrect"; true -> "Catch all" end)'

Catch all


In [ ]:
# with
# 特殊形式の with/1 はネストされた case/2 文やきれいにパイプできない状況に便利
# with/1 式はキーワード, ジェネレータ, そして式から成り立っている
# ジェネレータについてはリスト内包表記のところで詳しく述べる
# `<-` の右側と左側を比べるのにパターンマッチングが使われる
!elixir -e 'user=%{first: "Sean", last: "Callan"}; IO.inspect user'
!elixir -e 'user=%{first: "Sean", last: "Callan"}; with {:ok, first} <- Map.fetch(user, :first), {:ok, last} <- Map.fetch(user, :last), do: IO.puts last <> ", " <> first'

%{first: "Sean", last: "Callan"}
Callan, Sean


In [ ]:
# シェルワンライナーが長いのでファイルにする
%%writefile temp.exs
user=%{first: "Sean", last: "Callan"}
with {:ok, first} <- Map.fetch(user, :first), 
     {:ok, last} <- Map.fetch(user, :last), 
     do: IO.puts last <> ", " <> first

Overwriting temp.exs


In [ ]:
!elixir temp.exs

In [ ]:
# 式がマッチに失敗した場合
# Map.fetch が失敗して :error を返し、first が設定されずプログラムが止まる
%%writefile temp.exs
user = %{first: "doomspork"}
with {:ok, first} <- Map.fetch(user, :first), 
     {:ok, last} <- Map.fetch(user, :last), 
     do: IO.puts last <> ", " <> first

Overwriting temp.exs


In [ ]:
!elixir temp.exs

In [ ]:
# with/1 で else が使える
%%writefile temp.exs
import Integer
m = %{a: 1, c: 3}
a =
    with {:ok, number} <- Map.fetch(m, :a),
        true <- is_even(number) do
        IO.puts "#{number} divided by 2 is #{div(number, 2)}"
        :even
    else
        :error ->
        IO.puts("We don't have this item in map")
        :error
        _ ->
        IO.puts("It is odd")
        :odd
    end
IO.inspect a

Overwriting temp.exs


In [ ]:
!elixir temp.exs

We don't have this item in map
:error


# 関数 Function

In [ ]:
# 関数型言語では、関数は第一級オブジェクト first class object である
# ここでは無名関数、名前付き関数、アリティ、パターンマッチング、プライベート関数、ガード、デフォルト引数について学ぶ

# 無名関数 anonymous function
# fn end のキーワードを用い、 引数 `->` 関数定義 の形で定義する

%%writefile temp.exs
sum = fn (a, b) -> a + b end
IO.puts sum.(2, 3)

Overwriting temp.exs


In [ ]:
!elixir temp.exs

5


In [ ]:
# シェルワンライナーで書いてみる
!elixir -e 'sum=fn(a,b)->a+b end;IO.puts sum.(2,3)'

5


In [ ]:
# elixir では通常関数定義に省略記号 & を使う (キャプチャ演算子)
!elixir -e 'sum =  &(&1 + &2); IO.puts sum.(2, 3)'

5


---
質問 無名関数に引数を渡して結果を得るのはどうやるのか

&(&1 + &2).(2, 3) として出来なかった。 => 出来た。

!elixir -e 'IO.puts ((&(&1 + &2)).(2,3))'

In [ ]:
!elixir -e 'IO.puts ((fn (a,b) -> a + b end).(2,3))'

5


In [ ]:
!elixir -e 'IO.puts ((&(&1 + &2)).(2,3))'

5


In [ ]:
# 関数定義にパターンマッチングが使える
%%writefile temp.exs
handle_result = fn
  {:ok, _result} -> IO.puts "Handling result..."
  {:ok, _} -> IO.puts "This would be never run as previous will be matched beforehand."
  {:error} -> IO.puts "An error has occurred!"
end

some_result = 1

handle_result.({:ok, some_result}) #=> Handling result...
handle_result.({:error}) #=> An error has occured!

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Handling result...
An error has occurred!


In [ ]:
# 名前付き関数
# 名前付き関数はモジュール内部で def キーワードを用いて定義する
%%writefile temp.exs
defmodule Greeter do
  def hello(name) do
    "Hello, " <> name
  end
end

IO.puts Greeter.hello("Sean")

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean


In [ ]:
# 次のような書き方もできる do: を使う
%%writefile temp.exs
defmodule Greeter do
  def hello(name), do: "Hello, " <> name
end
IO.puts Greeter.hello("Sean")

Writing temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean


In [ ]:
# 実験 シェルワンライナーで出来るか
!elixir -e 'defmodule Greeter do def hello(name) do "Hello, " <> name end end;IO.puts Greeter.hello("Sean")'

Hello, Sean


In [ ]:
# 実験 シェルワンライナーで `, do:` 構文が使えるか
!elixir -e 'defmodule Greeter do def hello(name),do: "Hello, " <> name end;IO.puts Greeter.hello("Sean")'

Hello, Sean


In [ ]:
# 再帰
%%writefile temp.exs
defmodule Length do
  def of([]), do: 0
  def of([_ | tail]), do: 1 + of(tail)
end

IO.puts Length.of []
IO.puts Length.of [1, 2, 3]

Overwriting temp.exs


In [ ]:
!elixir temp.exs

0
3


In [ ]:
# アリティとは関数の引数の数
# 引数の数が違えば別の関数
%%writefile temp.exs
defmodule Greeter2 do
  def hello(), do: "Hello, anonymous person!"   # hello/0
  def hello(name), do: "Hello, " <> name        # hello/1
  def hello(name1, name2), do: "Hello, #{name1} and #{name2}"  # hello/2
end

IO.puts Greeter2.hello()
IO.puts Greeter2.hello("Fred")
IO.puts Greeter2.hello("Fred", "Jane")

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, anonymous person!
Hello, Fred
Hello, Fred and Jane


In [ ]:
# 関数とパターンマッチング
%%writefile temp.exs
defmodule Greeter1 do
  def hello(%{name: person_name}) do
    IO.puts "Hello, " <> person_name
  end
end

fred = %{
name: "Fred",
age: "95",
favorite_color: "Taupe"
}

IO.puts Greeter1.hello(fred) #=> Hello, fred になる
#IO.puts Greeter1.hello(%{age: "95", favorite_color: "Taupe"}) #=> (FunctionClauseError) no function clause matching in Greeter1.hello/1    

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Fred
ok


In [ ]:
# Fredの名前を person_name にアサインしたいが、人物マップ全体の値も保持したいという場合
# マップを引数にすれば、別々の変数に格納することができる
%%writefile temp.exs
defmodule Greeter2 do
  def hello(%{name: person_name} = person) do
    IO.puts "Hello, " <> person_name
    IO.inspect person
  end
end

fred = %{
    name: "Fred",
    age: "95",
    favorite_color: "Taupe"
}

Greeter2.hello(fred)
IO.puts("")
Greeter2.hello(%{name: "Fred"})
IO.puts("")
# Greeter2.hello(%{age: "95", favorite_color: "Taupe"}) #=> (FunctionClauseError) no function clause matching in Greeter2.hello/1  

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Fred
%{age: "95", favorite_color: "Taupe", name: "Fred"}

Hello, Fred
%{name: "Fred"}



In [ ]:
# %{name: person_name} と person の順序を入れ替えても、それぞれがfredとマッチングするので同じ結果となる
# 変数とマップを入れ替えてみる
# それぞれがパターンマッチしているので結果は同じになる
%%writefile temp.exs
defmodule Greeter3 do
  def hello(person = %{name: person_name}) do
    IO.puts "Hello, " <> person_name
    IO.inspect person
  end
end

fred = %{
    name: "Fred",
    age: "95",
    favorite_color: "Taupe"
}

Greeter3.hello(fred)
IO.puts("")
Greeter3.hello(%{name: "Fred"})

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Fred
%{age: "95", favorite_color: "Taupe", name: "Fred"}

Hello, Fred
%{name: "Fred"}


In [ ]:
# プライベート関数
# プライベート関数は defp を用いて定義する
# そのモジュール自身の内部からのみ呼び出すことが出来る
%%writefile temp.exs
defmodule Greeter do
  def hello(name), do: phrase() <> name
  defp phrase, do: "Hello, "
end

IO.puts Greeter.hello("Sean") #=> "Hello, Sean"

# IO.puts Greeter.phrase #=> (UndefinedFunctionError) function Greeter.phrase/0 is undefined or private

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean


In [ ]:
# ガード
%%writefile temp.exs
defmodule Greeter do
  def hello(names) when is_list(names) do
    names
    |> Enum.join(", ")
    |> hello
  end

  def hello(name) when is_binary(name) do
    phrase() <> name
  end

  defp phrase, do: "Hello, "
end

IO.puts Greeter.hello ["Sean", "Steve"]
IO.puts Greeter.hello "Bill"

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean, Steve
Hello, Bill


---
質問 Elixir のガードは Haskell のガードと同じか?

In [ ]:
# デフォルト引数
# デフォルト値が欲しい場合、引数 \\ デフォルト値の記法を用いる
%%writefile temp.exs
defmodule Greeter do
  def hello(name, language_code \\ "en") do
    phrase(language_code) <> name
  end

  defp phrase("en"), do: "Hello, "
  defp phrase("es"), do: "Hola, "
end

IO.puts Greeter.hello("Sean", "en")
IO.puts Greeter.hello("Sean")
IO.puts Greeter.hello("Sean", "es")

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean
Hello, Sean
Hola, Sean


In [ ]:
# ガードとデフォルト引数を組み合わせる場合
# 混乱を避けるためデフォルト引数を処理する定義を先に置く
%%writefile temp.exs
defmodule Greeter do
  def hello(names, language_code \\ "en")

  def hello(names, language_code) when is_list(names) do
    names
    |> Enum.join(", ")
    |> hello(language_code)
  end

  def hello(name, language_code) when is_binary(name) do
    phrase(language_code) <> name
  end

  defp phrase("en"), do: "Hello, "
  defp phrase("es"), do: "Hola, "
end

IO.puts Greeter.hello ["Sean", "Steve"] #=> "Hello, Sean, Steve"

IO.puts Greeter.hello ["Sean", "Steve"], "es" #=> "Hola, Sean, Steve"

IO.puts Greeter.hello "Bob", "es"

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello, Sean, Steve
Hola, Sean, Steve
Hola, Bob


In [ ]:
# パイプライン演算子
# パイプライン演算子 `|>` はある式の結果を別の式に渡す
# 関数のネストを理解しやすくするためのもの

# 文字列をトークン化する、単語に分ける
!elixir -e 'IO.inspect "Elixir rocks" |> String.split()'

["Elixir", "rocks"]


In [ ]:
!elixir -e 'IO.inspect "Elixir rocks" |> String.upcase() |> String.split()'

["ELIXIR", "ROCKS"]


In [ ]:
# パイプラインを使う場合に関数の括弧は省略せずには入れた方がわかりやすい
!elixir -e 'IO.inspect "elixir" |> String.ends_with?("ixir")'

true


# モジュール

---
質問 いままで IO.puts とか一々モジュール名を付けていたが、elixir ではこれが普通なのか?

関数を作る際に一々モジュールを作成していたがあれで既存のモジュールに付け加えられているのか?

In [ ]:
# モジュールの基本的な例
%%writefile temp.exs
defmodule Example do
  def greeting(name) do
    "Hello #{name}."
  end
end

IO.puts Example.greeting "Sean"

Writing temp.exs


In [ ]:
!elixir temp.exs

Hello Sean.


In [ ]:
# モジュールはネストする事ができる
%%writefile temp.exs
defmodule Example.Greetings do
  def morning(name) do
    "Good morning #{name}."
  end

  def evening(name) do
    "Good night #{name}."
  end
end

IO.puts Example.Greetings.morning "Sean"

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Good morning Sean.


In [ ]:
# モジュールの属性
# モジュール属性は Elixir では一般に定数として用いられる
# Elixirには予約されている属性がある
# moduledoc — 現在のモジュールにドキュメントを付ける
# doc — 関数やマクロについてのドキュメント管理
# behaviour — OTPまたはユーザが定義した振る舞い(ビヘイビア)に用いる
%%writefile temp.exs
defmodule Example do
  @greeting "Hello"

  def greeting(name) do
    ~s(#{@greeting} #{name}.)
  end
end

IO.puts Example.greeting "tak"

Overwriting temp.exs


In [ ]:
!elixir temp.exs

Hello tak.


In [15]:
# 構造体 struct 
# 構造体は定義済みのキーの一群とデフォルト値を持つマップである
# 定義するには defstruct を用いる
%%writefile temp.exs
defmodule Example.User do
  defstruct name: "Sean", roles: []
end

defmodule Main do
  IO.inspect %Example.User{}
  IO.inspect %Example.User{name: "Steve"}
  IO.inspect %Example.User{name: "Steve", roles: [:manager]}
end

Overwriting temp.exs


In [16]:
!elixir temp.exs

%Example.User{name: "Sean", roles: []}
%Example.User{name: "Steve", roles: []}
%Example.User{name: "Steve", roles: [:manager]}


In [ ]:
# 構造体の更新
%%writefile temp.exs
defmodule Example.User do
  defstruct name: "Sean", roles: []
end

defmodule Main do
  steve = %Example.User{name: "Steve"}
  IO.inspect %{steve | name: "Sean"}
  IO.inspect steve
end

In [30]:
!elixir temp.exs

%Example.User{name: "Sean", roles: []}
%Example.User{name: "Steve", roles: []}


In [31]:
# 構造体の更新とマッチング
%%writefile temp.exs
defmodule Example.User do
  defstruct name: "Sean", roles: []
end

defmodule Main do
  steve = %Example.User{name: "Steve"}
  sean = %{steve | name: "Sean"}
  IO.inspect %{name: "Sean"} = sean
end

Overwriting temp.exs


In [32]:
!elixir temp.exs

%Example.User{name: "Sean", roles: []}


In [42]:
# inspect の出力を変える
%%writefile temp.exs
defmodule Example.User do
  # @derive {Inspect, only: [:name]}
  @derive {Inspect, except: [:roles]}
  defstruct name: "Sean", roles: []
end

defmodule Main do
  steve = %Example.User{name: "Steve"}
  sean = %{steve | name: "Sean"}
  IO.inspect %{name: "Sean"} = sean
end

Overwriting temp.exs


In [43]:
!elixir temp.exs

#Example.User<name: "Sean", ...>


In [44]:
# コンポジション(Composition)
# コンポジションを用いてモジュールや構造体に既存の機能を追加する

# alias モジュール名をエイリアスする

%%writefile temp.exs

defmodule Sayings.Greetings do
  def basic(name), do: "Hi, #{name}"
end

defmodule Example do
  alias Sayings.Greetings
  def greeting(name), do: Greetings.basic(name)
end

IO.puts Example.greeting "Bob!!"

# aliasを使わない場合

# defmodule Example do
#   def greeting(name), do: Sayings.Greetings.basic(name)
# end

Overwriting temp.exs


In [45]:
!elixir temp.exs

Hi, Bob!!


In [46]:
# 別名で alias したい時は `:as` を使う
%%writefile temp.exs
defmodule Sayings.Greetings do
  def basic(name), do: "Hi, #{name}"
end

defmodule Example do
  alias Sayings.Greetings, as: Hi
  def print_message(name), do: Hi.basic(name)
end

IO.puts Example.print_message "Chris!!"

Overwriting temp.exs


In [47]:
!elixir temp.exs

Hi, Chris!!


In [ ]:
# 複数のモジュールを一度に alias する

# defmodule Example do
#   alias Sayings.{Greetings, Farewells}
# end

In [50]:
# import
# 関数を取り込みたいという場合には、 import を使う
!elixir -e 'import List; IO.inspect last([1,2,3])'

3


In [62]:
# フィルタリング
# import のデフォルトでは全ての関数とマクロが取り込まれるが、 :only や :except でフィルタすることができる
# アリティを付ける必要がある
%%writefile temp.exs
import List, only: [last: 1]
IO.inspect last([1,2,3])
# IO.inspect first([1,2,3]) #=>  (CompileError) temp.exs:3: undefined function first/1 (there is no such import)

Overwriting temp.exs


In [63]:
!elixir temp.exs

3


In [ ]:
# import には :functions と :macros という2つの特別なアトムもありるこれらはそれぞれ関数とマクロのみを取り込む

# import List, only: :functions
# import List, only: :macros

In [ ]:
# require と import の違いがわからない
# まだロードされていないマクロを呼びだそうとすると、Elixirはエラーを発生させる
# とのこと

# defmodule Example do
#   require SuperMacros
# 
#   SuperMacros.do_stuff
# end

# いまここ

In [ ]:
# use
use マクロを用いることで他のモジュールを利用して現在のモジュールの定義を変更することができる コード上で use を呼び出すと、
実際には提供されたモジュールに定義されている __using__/1 コールバックを呼び出しる __using__/1 
マクロの結果はモジュールの定義の一部になりる この動作に対する理解を深めるために簡単な例を見ましょう:

defmodule Hello do
  defmacro __using__- do
    quote do
      def hello(name), do: "Hi, #{name}"
    end
  end
end
ここでは hello/1 関数を定義する __using__/1 コールバックを定義した Hello モジュールを作りました。 
この新しいコードを試すために新しいモジュールを作ります:

defmodule Example do
  use Hello
end
IExでこのコードを試して見ると Example モジュールで hello/1 を使えるのがわかりる

Example.hello("Sean")
"Hi, Sean"
ここで use が Hello の __using__/1 コールバックを呼び出して、結果のコードをモジュールに追加しる 基本的な例を見せたので、ここからはこのコードを変更して __using__/1 にオプションをサポートする方法を見てみましょう。 greeting オプションを追加します:

defmodule Hello do
  defmacro __using__(opts) do
    greeting = Keyword.get(opts, :greeting, "Hi")

    quote do
      def hello(name), do: unquote(greeting) <> ", " <> name
    end
  end
end
新しく作った greeting オプションを含むために Example モジュールを更新します:

defmodule Example do
  use Hello, greeting: "Hola"
end
IExで試して見ると挨拶が変わるのを確認できる

iex> Example.hello("Sean")
"Hola, Sean"
これらは use がどうやって動作するのかを説明する簡単な例でしたが、
これはElixirのツールボックスで信じられないほどに強力なツールです。 
Elixirを学び続けたら use をあっちこっちで見ることになるでしょう。
かならず見ることになりそうな例をひとつあげれば、 use ExUnit.Case, async: true です。

注意: quote 、 alias 、 use 、 require はメタプログラミングで使用してたマクロです。

Caught a mistake or want to contribute to the lesson? Edit this lesson on GitHub!
PREVIOUS LESSON
パイプライン演算子
パイプライン演算子(|>)はある式の結果を別の式に1つ目の引数として渡しる

NEXT LESSON
Mix
私たちがElixirの深海へと潜れるようになる前に、まずmixについて学ぶ必要がありる
Rubyに詳しければ、mixとはBundlerとRubyGems、そしてRakeが組み合わさったものです。
あらゆるElixirプロジェクトに欠かせない部分であり、このレッスンではその素晴らしい機能のうち、
ほんの一部を探検していきるあなたの環境でmixが提供する全ての機能を知るにはmix help
を実行してください。

今まではもっぱら制限のある iex の内部で取り組んできました。何か実際に動くものを作るには、
コードを複数のファイルに分けて効率的に管理する必要がありるそしてmixは複数の
プロジェクトでそうした開発を支える



# Mix

In [ ]:
Elixir School
        
Mix
和訳が古くなっている可能性がありる
いくつかの軽微な変更が翻訳元の文書に適用されている
私たちがElixirの深海へと潜れるようになる前に、まずmixについて学ぶ必要がありるRubyに詳しければ、mixとはBundlerとRubyGems、そしてRakeが組み合わさったものです。あらゆるElixirプロジェクトに欠かせない部分であり、このレッスンではその素晴らしい機能のうち、ほんの一部を探検していきるあなたの環境でmixが提供する全ての機能を知るにはmix helpを実行してください。

今まではもっぱら制限のある iex の内部で取り組んできました。何か実際に動くものを作るには、コードを複数のファイルに分けて効率的に管理する必要がありるそしてmixは複数のプロジェクトでそうした開発を支える

目次
新しいプロジェクト
対話的な方法
コンパイル
依存関係を管理する
環境
新しいプロジェクト 
新しいElixirプロジェクトを立ち上げる準備ができたら、mixのmix newコマンドで簡単に行えるこれにより、プロジェクトのフォルダ構成と必要なボイラープレート(決まりきったソースコード断片)が生成されるとても分かりやすいですね、始めましょう:

Copy
$ mix new example
実行結果から、mixがディレクトリと一連のボイラープレートファイルを作っていることがわかります:

Copy
* creating README.md
* creating .gitignore
* creating .formatter.exs
* creating mix.exs
* creating lib
* creating lib/example.ex
* creating test
* creating test/test_helper.exs
* creating test/example_test.exs
このレッスンでは mix.exs に焦点を合わせていきるアプリケーションや依存関係、環境、そしてバージョンについて設定を行うところです。このファイルをお好きなエディタで開いてください。このような感じに見えるはずです(コメントは簡潔さのために除いています):

Copy
defmodule Example.Mix do
  use Mix.Project

  def project do
    [
      app: :example,
      version: "0.1.0",
      elixir: "~> 1.5",
      start_permanent: Mix.env() == :prod,
      deps: deps()
    ]
  end

  def application do
    [
      extra_applications: [:logger]
    ]
  end

  defp deps do
    []
  end
end
最初に見える項目は project です。ここでアプリケーションの名前(app)を定義し、そのバージョン(version)と用いるElixirのバージョン(elixir)と、最後に依存関係(deps)を記述しる

application の項は、次項で扱うアプリケーションファイルの生成時に使用しる

対話的な方法 
アプリケーションの機能や設定を利用できる環境の中で iex を使う必要があるかもしれません。ありがたいことに、mixを使えば簡単です。コンパイルされたアプリケーションとともに新しい iex セッションを開始することができます:

Copy
$ cd example
$ iex -S mix
このように iex を立ち上げると、アプリケーションと依存関係が現在の実行環境へ読み込まれる

コンパイル 
mixは賢く、必要に応じてコードの変更をコンパイルしてくれますが、それでもプロジェクトを明示的にコンパイルする必要があるかもしれません。この項ではプロジェクトのコンパイル方法と、コンパイルが何をしているのかについて扱いる

mixプロジェクトをコンパイルするにはプロジェクト直下のディレクトリでただmix compileを実行するだけで済みる

注記：プロジェクトのmix taskはプロジェクトのrootディレクトリからのみ実行可能です。rootディレクトリ以外から実行する場合は、グローバルmix taskのみが実行可能です。

Copy
$ mix compile
私達のプロジェクトはまだ中身があまりないので出力結果はそれほど面白くありませんが、無事に完了するはずです:

Copy
Compiled lib/example.ex
Generated example app
プロジェクトをコンパイルする際、mixは生成物のために _build ディレクトリを作りる _build 内部を覗けば、コンパイルされたアプリケーションが example.app として見えるでしょう。

依存関係を管理する 
私達のプロジェクトはまだ依存関係を持っていませんが、それもすぐでしょうから、次へと進み、依存関係を定義し取り込む方法を扱いる

新しい依存関係を追加するには、まずそれを mix.exs の deps 内に追加する必要がありる依存関係のリストは2つの必須な値(パッケージ名のアトムと、バージョンを表す文字列)と1つの任意的な値(オプション)を持つタプルで成り立ちる

実例として、phoenix_slimのようなプロジェクトの依存関係を見てみましょう:

Copy
def deps do
  [
    {:phoenix, "~> 1.1 or ~> 1.2"},
    {:phoenix_html, "~> 2.3"},
    {:cowboy, "~> 1.0", only: [:dev, :test]},
    {:slime, "~> 0.14"}
  ]
end
上記を見てお気づきかと思いますが、 cowboy の依存は開発時とテスト時にのみ必要です。

依存関係を定義したら、あとは最後の一歩、依存しているパッケージの取り込みです。これはbundle installに似たものです:

Copy
$ mix deps.get
完了です！プロジェクトの依存関係を定義し、取り込みを行いました。これで、しかるべき時に依存関係を追加する準備ができました。

環境 
mixは、Bundlerにとても似て、様々な環境に対応しているmixは最初から3つの環境で動作するように構成されています:

:dev - 初期状態での環境。
:test - mix testで用いられる環境。次のレッスンでさらに見ていきる
:prod - アプリケーションを製品に出荷するときに用いられる環境。
現在の環境は Mix.env で取得することができる期待される通り、この環境は MIX_ENV 環境変数によって変更することができます:

Copy
$ MIX_ENV=prod mix compile
Caught a mistake or want to contribute to the lesson? Edit this page on GitHub!
← モジュール
Sigils →
bg bn de en es fr gr id it ja ko ms no pl pt ru sk th tr uk vi zh-hans zh-hant
Elixir 1.10.1 - Erlang/OTP 22.0 [erts-10.5.3]
Menu
プログラミング言語Elixirのレッスン
© 2021 Sean Callan All rights reserved.

Toggle